# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin
import pickle



nltk.download(['punkt', 'wordnet'])

In [ ]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterResponse", engine)
df = df.iloc[:20,:]
df.head()

In [ ]:
df.shape

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return tokens

In [ ]:
# vect = CountVectorizer(tokenizer=tokenize)
# tfidf = TfidfTransformer()
# clf = MultiOutputClassifier(RandomForestClassifier())

# # train classifier
# X_train_counts = vect.fit_transform(X_train)
# X_train_tfidf = tfidf.fit_transform(X_train_counts)
# clf.fit(X_train_tfidf, y_train)

# # predict on test data
# X_test_counts = vect.transform(X_test)
# X_test_tfidf = tfidf.transform(X_test_counts)
# y_pred = clf.predict(X_test_tfidf)

# accuracy = (y_pred == y_test).mean().mean()
# accuracy

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [ ]:
pipeline.get_params()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X = df['message']
y = df.iloc[:,4:].astype('int64')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X.iloc[:50], y.iloc[:50,:], random_state=42)

model = pipeline
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = (y_pred == y_test).mean().mean()
print("Accuracy:", accuracy)

In [ ]:
X = df['message']
y = df.iloc[:,4:].astype('int64')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X.iloc[:50], y.iloc[:50,:], random_state=42)
list(np.unique(y))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
for i in range(len(df.columns[5:])):
    print(df.columns[5+i])
    print(classification_report(y_test.to_numpy()[:, 0], y_pred[:, 0]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'clf__estimator__bootstrap': (True, False)
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3) #added verbose=3

In [ ]:
pipeline.get_params()

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
def display_results(cv, y_test, y_pred):
    accuracy = (y_pred == y_test).mean()

    print("Accuracy:", accuracy)
    print("\nOverall Accuracy:", accuracy.mean())
    print("\nBest Parameters:", cv.best_params_)
    

    
def main():
    X = df['message']
    y = df.iloc[:,4:].astype('int64')
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    # X_train, X_test, y_train, y_test = train_test_split(X.iloc[:50], y.iloc[:50,:], random_state=42)

    model = build_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    display_results(model, y_test, y_pred)
    
    
main()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
class MessageLength(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return pd.DataFrame(pd.Series(X).apply(lambda x: len(x)).values)
        # return X.apply(lambda x: len(x)) didn't work. 
        # This is because the Custom Transformer returns only one row instead of 19584.
        # The custom transformer is in parallel to the tfidftransformer, So the number 
        # of observations from both the transformer should be same.

pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 2))), # ngram_range=(1, 2) from best param
            ('tfidf', TfidfTransformer())
        ])),

        ('txt_len', MessageLength())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier(bootstrap = False))) # bootstrap = False from best param
])
     

model = pipeline
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = (y_pred == y_test).mean().mean()
print("Accuracy:", accuracy)

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(model, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [1]:
import sys
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin
import pickle


def load_data(database_filepath):
    engine = create_engine('sqlite:///' + database_filepath)
    df = pd.read_sql("SELECT * FROM DisasterResponse", engine)
    df = df.iloc[:50,:]
    X = df['message']
    Y = df.iloc[:,4:].astype('int64')
    category_names = list(np.unique(Y))
    
    return X, Y, category_names


def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return tokens


def build_model():
    class MessageLength(BaseEstimator, TransformerMixin):
        def fit(self, X, y=None):
            return self
        


        def transform(self, X):
            return pd.DataFrame(pd.Series(X).apply(lambda x: len(x)).values)
            # return X.apply(lambda x: len(x)) didn't work. 
            # This is because the Custom Transformer returns only one row instead of 19584.
            # The custom transformer is in parallel to the tfidftransformer, So the number 
            # of observations from both the transformer should be same.

    parameters = {
        'clf__estimator__criterion': ('gini', 'entropy'),
        'clf__estimator__bootstrap': (True, False)
    }
        
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('txt_len', MessageLength())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3) #added verbose=3
    
    return cv


def evaluate_model(cv, X_test, Y_test, category_names):
    Y_pred = cv.predict(X_test)
    accuracy = (Y_pred == Y_test).mean().mean()
    return accuracy


def save_model(model, model_filepath):
    pickle.dump(model, open(model_filepath, 'wb'))
    
    #with open(model_filepath, 'wb') as file:
    #    pickle.dump(model, file)
        
        
def main():
    database_filepath, model_filepath = 'DisasterResponse.db', 'classifier.pkl'
    print('Loading data...\n    DATABASE: {}'.format(database_filepath))
    X, Y, category_names = load_data(database_filepath)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

    print('Building model...')
    model = build_model()

    print('Training model...')
    model.fit(X_train, Y_train)

    print('Model Parameters...')
    print(model.best_params_)

    #model = model.best_estimator_

    print('Evaluating model...')
    print(evaluate_model(model, X_test, Y_test, category_names))

    print('Saving model...\n MODEL: {}'.format(model_filepath))
    save_model(model, model_filepath)
        
main()

Loading data...
    DATABASE: DisasterResponse.db
Building model...
Training model...
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__bootstrap=True, clf__estimator__criterion=gini ..


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__bootstrap=True, clf__estimator__criterion=gini, score=0.000, total=   7.2s
[CV] clf__estimator__bootstrap=True, clf__estimator__criterion=gini ..


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.1s remaining:    0.0s


[CV]  clf__estimator__bootstrap=True, clf__estimator__criterion=gini, score=0.000, total=   5.5s
[CV] clf__estimator__bootstrap=True, clf__estimator__criterion=gini ..


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   12.6s remaining:    0.0s


[CV]  clf__estimator__bootstrap=True, clf__estimator__criterion=gini, score=0.000, total=   6.6s
[CV] clf__estimator__bootstrap=True, clf__estimator__criterion=gini ..
[CV]  clf__estimator__bootstrap=True, clf__estimator__criterion=gini, score=0.000, total=   5.6s
[CV] clf__estimator__bootstrap=True, clf__estimator__criterion=gini ..
[CV]  clf__estimator__bootstrap=True, clf__estimator__criterion=gini, score=0.250, total=   4.9s
[CV] clf__estimator__bootstrap=True, clf__estimator__criterion=entropy 
[CV]  clf__estimator__bootstrap=True, clf__estimator__criterion=entropy, score=0.000, total=   5.0s
[CV] clf__estimator__bootstrap=True, clf__estimator__criterion=entropy 
[CV]  clf__estimator__bootstrap=True, clf__estimator__criterion=entropy, score=0.000, total=   5.0s
[CV] clf__estimator__bootstrap=True, clf__estimator__criterion=entropy 
[CV]  clf__estimator__bootstrap=True, clf__estimator__criterion=entropy, score=0.000, total=   5.1s
[CV] clf__estimator__bootstrap=True, clf__estimator

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  1.6min finished


Model Parameters...
{'clf__estimator__bootstrap': False, 'clf__estimator__criterion': 'entropy'}
Evaluating model...
0.9138888888888886
Saving model...
 MODEL: classifier.pkl


AttributeError: Can't pickle local object 'build_model.<locals>.MessageLength'